# 다중 회귀 
- 다중의 독립 변수가 존재하는 회귀 분석 
- 여러 개의 독립 변수가 복합적으로 종속 변수에 영향을 미치는 경우 다중 회귀 모형으로 데이터를 예측
- 다중 회귀에서 최적의 모델을 결정하기 위해 여러 가지의 방법이 존재 
- 모델이 복잡해지면 과대적합이 발생할 가능성이 있기 때문에 이를 방지하기 위해 다양한 규제 방식이 존재하고 규제에 따른 모델들이 존재 
    - 모델에서 규제 방식에 따라서 가중치를 제한
    - 독립 변수에서의 가중치가 0이 되는 차원(컬럼 | 피쳐)들이 생성 
    - 규제를 강하게 걸면 가중치의 절대치가 줄어들고 0에 가까워진다. 

## 릿지 
- 최소제곱 적합식의 수축 패널티라 불리는 항에 L2 패널티를 추가한것 

- 매개변수 
    - alpha
        - 기본값 : 1.0
        - 규제의 강도 -> 클수록 회귀 계수가 작아지고 과적합의 방지, 과소적합 위험
        - 해당 모델의 중요한 매개변수
    - solver
        - 기본값 : 'auto'
        - 해를 구하는 방법 
        - 데이터의 크기 / 희소성에 따라 적합한 solver를 선택 
        - 'auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'
            - svd : 특이값 분해, 다중공선성 있고 데이터의 개수가 작거나 중간 정도 
            - cholesky : 정규방정식, 데이터의 개수가 작거나 중간 정도 
            - lsqr : 반복 최소제곱, 대규모 데이터중 희소/밀집 데이터
            - sparse_cg : 공액 기울기법, 대규모 데이터 중 희소 데이터 
            - sag : 확률적 평균 검사, 데이터의 행의 수가 열의 수보다 월등히 많은 경우
            - saga : sag 확장, 대규모 희소, ElasticNet 가능
    - tol
        - 기본값 : 0.001 (1e-03)
        - 수렴의 판단 기준, 작을수록 정밀, 속도적인 면에서 느려질수 있다. 
    - max_iter
        - 기본값 : None
        - 최적화 될때까지 최대 반복 횟수.(데이터 크거나 수치가 불안정 경우에 필요)
    
- 속성 
    - coef_
        - 회귀 계수를 출력 (규제로 인해서 선형 회귀에 비해 값들이 작게 출력)
    - n_iter_
        - solver가 반복한 횟수 출력 


In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt

In [2]:
diabetes = load_diabetes()

In [3]:
df = pd.DataFrame(diabetes['data'], columns = diabetes['feature_names'])
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641


In [4]:
alpha = np.logspace(-3, 1, 5)
alpha

array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01])

In [ ]:
# alpha 값에 따라 회귀계수가 어떻게 변화하는가?
# 반복 실행할때 마다 회귀계수를 추가해주는 빈 리스트 
data = []

for a in alpha:
    # print(a)
    # Ridge class 생성 시 alpha 매개변수에 a를 대입
    ridge = Ridge(alpha=a)
    # 모델 학습 
    ridge.fit(df.values, diabetes['target'])
    # 학습 된 모델에서 회귀계수를 출력하여 data에 추가 
    data.append(
        ridge.coef_
    )
data

In [10]:
df_ridge =  pd.DataFrame(data, index = alpha, columns = df.columns)